In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,2,3,4"

In [3]:
import tensorflow as tf

# Check if a GPU is available
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available!")
    for gpu in gpu_devices:
        print(f"Device: {gpu.name}")
else:
    print("No GPU available")

# Check if CUDA is available
cuda_available = tf.test.is_built_with_cuda()
print(f"CUDA Available: {cuda_available}")

# Check cuDNN version
try:
    from tensorflow.python.framework import ops
    print(f"cuDNN Version: {tf.sysconfig.get_build_info()['cuda_version']}")
except Exception as e:
    print(f"Error fetching cuDNN version: {e}")

No GPU available
CUDA Available: False
Error fetching cuDNN version: 'cuda_version'


In [4]:
IMG_SHAPE = (128, 128, 3)  # Height, Width, Channels

In [5]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [6]:
import tensorflow as tf
import os
import pandas as pd

# Step 1: Define paths
image_dir = "./celeba_Orig/images/"
attributes_file = "./celeba_Orig/list_attr_celeba.txt"

# Step 2: Load the attribute file
attributes = pd.read_csv(attributes_file, delim_whitespace=True, skiprows=1)
attributes.reset_index(inplace=True)  # Reset index to access the filename column
attributes.rename(columns={'index': 'image_name'}, inplace=True)

# Step 3: Filter for Specific Attributes
# Keep only the relevant attributes
attributes = attributes[['image_name', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young']]

# Step 4: Select a Subset of Images (1/10 of the dataset)
subset_fraction = 0.1
subset_size = int(len(attributes) * subset_fraction)
attributes = attributes.sample(n=subset_size, random_state=42).reset_index(drop=True)

# Step 5: Define train, test, valid splits
# Calculate split sizes based on the subset
num_train = int(0.8 * subset_size)  # 80% for training
num_valid = int(0.1 * subset_size)  # 10% for validation

train_data = attributes[:num_train]
valid_data = attributes[num_train:num_train + num_valid]
test_data = attributes[num_train + num_valid:]

# Step 6: Create a function to load and preprocess images
def load_image(img_name, label):
    # Ensure img_name is a string by decoding the Tensor
    img_name = tf.strings.join([image_dir, img_name])
    img = tf.io.read_file(img_name)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode JPEG
    img = tf.image.resize(img, [128, 128])  # Resize to desired size
    img = img / 255.0  # Normalize to [0, 1]
    return img, label


# Step 7: Create a function to generate a TensorFlow dataset
def create_dataset(dataframe):
    image_names = dataframe['image_name'].values
    labels = dataframe.iloc[:, 1:].values  # All columns except 'image_name'
    
    # Convert labels from -1,1 to 0,1 (for binary classification)
    labels = (labels + 1) // 2  # Convert -1, 1 to 0, 1
    
    # Labels are already binary (0 or 1), no need for one-hot encoding
    dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))
    dataset = dataset.map(lambda x, y: load_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

# Step 8: Create train, test, and validation datasets
raw_train = create_dataset(train_data).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
raw_valid = create_dataset(valid_data).batch(64).prefetch(tf.data.AUTOTUNE)
raw_test = create_dataset(test_data).batch(64).prefetch(tf.data.AUTOTUNE)

# Step 9: Verify the dataset
for images, labels in raw_train.take(1):
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")


C:\Users\Gordts-De Laender\AppData\Local\Temp\ipykernel_13556\3508556547.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  attributes = pd.read_csv(attributes_file, delim_whitespace=True, skiprows=1)


Image batch shape: (64, 128, 128, 3)
Label batch shape: (64, 5)


In [7]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5, activation='sigmoid')  # Use sigmoid for multi-label classification

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pickle

# Define validation steps (can be adjusted as needed)
validation_steps = 31 #Number of validation images 2000 /Batch size 64
initial_epochs = 20

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',       # Monitor validation loss
    patience=3,               # Patience of 3 epochs
    restore_best_weights=True # Restore best weights
)

# Define learning rate scheduler callback
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',      # Monitor validation loss
    factor=0.5,              # Reduce learning rate by 50%
    patience=2,              # Wait for 2 epochs without improvement before reducing
    min_lr=1e-6              # Minimum learning rate to avoid too small learning rate
)

# Evaluate the model on the validation dataset
loss0, accuracy0 = model.evaluate(raw_valid, steps=validation_steps)

# Train the model with both callbacks
history = model.fit(
    raw_train,
    epochs=initial_epochs,
    validation_data=raw_valid,
    callbacks=[early_stopping, lr_scheduler]  # Include both callbacks
)

# Save the model
model.save('my_model.h5')  # Saves the model in HDF5 format

# Save history to a file
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

25/31 [=======================>......] - ETA: 3s - loss: 0.7688 - binary_accuracy: 0.4614 

In [ ]:
import matplotlib.pyplot as plt

# Extract the loss and accuracy history
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define the number of epochs
epochs = range(1, len(acc) + 1)

# Plot Accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model('my_model.h5')

# Define a function to preprocess the input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # Resize the image
    img_array = image.img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Define categories
categories = ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young']

# Directory containing images
image_directory = '.'  # Replace with the path to your image directory
image_paths = [os.path.join(image_directory, fname) for fname in os.listdir(image_directory) if fname.endswith(('.png', '.jpg', '.jpeg'))]

# Set up grid size
grid_size = 3
fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
axes = axes.flatten()  # Flatten axes for easy iteration

# Iterate through images and display predictions
for i, ax in enumerate(axes):
    if i < len(image_paths):
        img_path = image_paths[i]
        
        # Preprocess and predict
        img = preprocess_image(img_path)
        predictions = model.predict(img)
        binary_predictions = (predictions >= 0.5).astype(int)
        
        # Display image
        img_display = image.load_img(img_path, target_size=(128, 128))
        ax.imshow(img_display)
        ax.axis('off')  # Hide axis
        
        # Display categories
        prediction_text = ", ".join([f"{cat}: {'Yes' if binary_predictions[0][j] == 1 else 'No'}" for j, cat in enumerate(categories)])
        ax.set_title(prediction_text, fontsize=10)
    else:
        ax.axis('off')  # Hide any unused grid cells

# Adjust layout and display the grid
plt.tight_layout()
plt.show()
